## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-13-18-56-06 +0000,nypost,Homeless criminal beat elderly Sikh worshipper...,https://nypost.com/2025/08/13/us-news/homeless...
1,2025-08-13-18-54-05 +0000,bbc,Europe expresses hope after call with Trump on...,https://www.bbc.com/news/articles/cpv0l9e187yo...
2,2025-08-13-18-50-41 +0000,startribune,Live: Pohlad family will not sell Minnesota Twins,https://www.startribune.com/live-pohlad-family...
3,2025-08-13-18-50-00 +0000,wsj,Bullish’s Stock Soars in Latest Test for IPO M...,https://www.wsj.com/finance/stocks/bullish-ipo...
4,2025-08-13-18-45-00 +0000,wsj,Trump Agrees on Ukraine Red Lines With Europe ...,https://www.wsj.com/world/trump-talks-ukraine-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,68
17,putin,21
22,will,16
18,summit,12
116,new,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
93,2025-08-13-15-07-51 +0000,nyt,Trump’s Affinity for Putin Will Be Tested at H...,https://www.nytimes.com/2025/08/13/us/politics...,134
63,2025-08-13-16-35-22 +0000,nypost,Trump says DC will be ‘beacon’ for other citie...,https://nypost.com/2025/08/13/us-news/trump-sa...,132
94,2025-08-13-15-03-41 +0000,nypost,Trump vows main goal of Putin Alaska summit is...,https://nypost.com/2025/08/13/world-news/trump...,130
243,2025-08-12-22-53-38 +0000,nypost,What to expect from Friday’s Trump-Putin summi...,https://nypost.com/2025/08/12/world-news/what-...,129
4,2025-08-13-18-45-00 +0000,wsj,Trump Agrees on Ukraine Red Lines With Europe ...,https://www.wsj.com/world/trump-talks-ukraine-...,128


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
93,134,2025-08-13-15-07-51 +0000,nyt,Trump’s Affinity for Putin Will Be Tested at H...,https://www.nytimes.com/2025/08/13/us/politics...
295,65,2025-08-12-19-21-00 +0000,wsj,The State Department released a long-delayed h...,https://www.wsj.com/politics/policy/state-depa...
162,52,2025-08-13-10-00-00 +0000,nypost,Sean Duffy opens up on spat with Elon Musk ove...,https://nypost.com/2025/08/13/us-news/sean-duf...
198,50,2025-08-13-04-49-45 +0000,nypost,National Guard takes to the streets of DC in s...,https://nypost.com/2025/08/13/us-news/national...
217,45,2025-08-13-01-44-11 +0000,latimes,"She was killed while walking to her car, famil...",https://www.latimes.com/california/story/2025-...
72,43,2025-08-13-16-25-00 +0000,wsj,New York Attorney General Letitia James is sui...,https://www.wsj.com/politics/policy/new-york-a...
298,42,2025-08-12-19-08-00 +0000,wsj,The White House plans to conduct a far-reachin...,https://www.wsj.com/politics/policy/white-hous...
193,41,2025-08-13-05-46-11 +0000,bbc,Israel bombards Gaza City as UK and allies dem...,https://www.bbc.com/news/articles/clyj0dd0qj9o...
78,39,2025-08-13-16-03-42 +0000,nypost,"Trump announces George Strait, Sylvester Stall...",https://nypost.com/2025/08/13/us-news/trump-an...
201,36,2025-08-13-04-00-51 +0000,nypost,Zelensky says Putin wants remainder of Ukraine...,https://nypost.com/2025/08/13/world-news/zelen...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
